In [7]:
import time
import urllib2
import datetime
from itertools import ifilter
from collections import Counter, defaultdict
import xml.etree.ElementTree as ET

from bs4 import BeautifulSoup
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
from pandas import HDFStore
import ads
from geopy.geocoders import Nominatim

%matplotlib inline

In [8]:
OAI = "{http://www.openarchives.org/OAI/2.0/}"
ARXIV = "{http://arxiv.org/OAI/arXiv/}"

df = pd.DataFrame(columns=("id", "created"))
base_url = "http://export.arxiv.org/oai2?verb=ListRecords&"
url = (base_url +
       "from=2015-01-01&until=2015-12-31&" +
       "metadataPrefix=arXiv&set=physics:astro-ph")

while True:
    print "fetching", url
    try:
        response = urllib2.urlopen(url)

    except urllib2.HTTPError, e:
        if e.code == 503:
            to = 30
            print "Got 503. Retrying after {0:d} seconds.".format(to)

            time.sleep(to)
            continue

        else:
            raise

    xml = response.read()

    root = ET.fromstring(xml)

    for record in root.find(OAI+'ListRecords').findall(OAI+"record"):
        arxiv_id = record.find(OAI+'header').find(OAI+'identifier')
        meta = record.find(OAI+'metadata')
        info = meta.find(ARXIV+"arXiv")
        created = info.find(ARXIV+"created").text
        created = datetime.datetime.strptime(created, "%Y-%m-%d")
        
        contents = {
                    'id': info.find(ARXIV+"id").text,
                    'created': created,
                    }

        df = df.append(contents, ignore_index=True)

    token = root.find(OAI+'ListRecords').find(OAI+"resumptionToken")
    if token is None or token.text is None:
        break

    else:
        url = base_url + "resumptionToken=%s"%(token.text)


fetching http://export.arxiv.org/oai2?verb=ListRecords&from=2015-01-01&until=2015-12-31&metadataPrefix=arXiv&set=physics:astro-ph
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1050080|1001
Got 503. Retrying after 30 seconds.
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1050080|1001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1050080|2001
Got 503. Retrying after 30 seconds.
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1050080|2001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1050080|3001
Got 503. Retrying after 30 seconds.
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1050080|3001
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1050080|4001
Got 503. Retrying after 30 seconds.
fetching http://export.arxiv.org/oai2?verb=ListRecords&resumptionToken=1050080|4001
fetching http://export.arxiv.org/oai2?verb=ListRecords

In [9]:
df = df[df['created'].dt.year == 2015]

In [10]:
df.to_csv('/Users/harshilkamdar/a-year-of-astronomy-arxiv.h5')

In [37]:
df = pd.read_csv('/Users/harshilkamdar/a-year-of-astronomy-arxiv.h5')
df = df.drop('Unnamed: 0', 1)
df_arxiv = df

In [51]:
papers = list(ads.SearchQuery(q='year:2010-2015 database:astronomy property:refereed', max_pages=2907))

KeyboardInterrupt: 

In [46]:
data = {
    "id": list(df['id'].values),
    "date": list(df['created'].values),
    "locations": []
}

In [47]:
arxiv_ids = df['id'].values
ads_ids = []
for paper in papers:
    ads_ids.append(paper.identifier[0])

KeyboardInterrupt: 

In [ ]:
print(ads_ids[0])
print(arxiv_ids[0])

In [33]:
query_string = ''
for i, arxiv_id in df['id'].iteritems():
    query_string += 'arxiv:%s OR ' % arxiv_id

In [34]:
query_string = query_string[:(len(query_string))-4]

In [43]:
print(papers[0])

NameError: name 'papers' is not defined

In [50]:
print(papers[5000].identifier)

[u'2014arXiv1411.1518V', u'2015ApJ...798..118V', u'2014arXiv1411.1518V', u'10.1088/0004-637X/798/2/118', u'arXiv:1411.1518', u'10.1088/0004-637X/798/2/118']
